# Семинар 2. Word2Vec/fastText и семантическое сходство (Colab-friendly)

**Цель семинара:** научиться работать с эмбеддингами слов и документов: считать **косинусную близость**, находить **похожие слова/тексты**, собирать **doc‑vector**, и сравнить это с TF‑IDF бейзлайном из семинара 1.

> Формат: короткие объяснения → маленькие примеры → один «мини‑проект» в конце.  
> Если что-то ломается — это не вы, это жизнь 🙂 (ниже есть фолбэки).

---

## План
1. Зачем эмбеддинги после TF‑IDF (1 слайд напоминания)
2. Математика: косинусная близость (ручной расчёт + код)
3. Мини‑эмбеддинги «с нуля»: co-occurrence → PMI/PPMI → SVD (прям руками)
4. Word2Vec на нашем корпусе (gensim): CBOW vs Skip‑gram
5. fastText и подсловные n‑граммы: почему это важно для русского
6. Документные вектора: среднее по словам и TF‑IDF‑взвешенное среднее
7. Мини‑проект: семантический поиск похожих твитов + сравнение с TF‑IDF
8. Бонус: классификация по doc‑vector vs TF‑IDF (честно сравним)

---


## Пакеты (один раз фиксируем)

⚠️ Если вы в Google Colab — этот блок обычно обязателен.  
Если локально — возможно, всё уже стоит.


In [ ]:
# === Установка зависимостей (Colab-friendly) ===
# Если вы запускаете локально и всё уже установлено — эту клетку можно пропустить/закомментировать.
!pip -q install gensim scikit-learn pandas numpy nltk

import re
import math
import random
import numpy as np
import pandas as pd

random.seed(42)
np.random.seed(42)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 51.6 MB/s eta 0:00:00


## 1) Почему эмбеддинги — это следующий шаг после TF‑IDF

TF‑IDF — честный бейзлайн, но:
- он **не знает**, что «машина» ≈ «автомобиль»
- плохо переносит **морфологию** (особенно русский)
- признаки разреженные и плохо **обобщают**

Эмбеддинги делают другое представление: **слово → плотный вектор**, а «смысл» превращается в **геометрию**.


## 2) Математика: косинусная близость

Косинусная близость двух векторов **u** и **v**:

$$
\cos(\theta) = \frac{\mathbf u \cdot \mathbf v}{\|\mathbf u\|\,\|\mathbf v\|}
$$

Интуиция:
- скалярное произведение **u·v** — «насколько сонаправлены»
- делим на длины — получаем **нормированное** сходство (важнее *направление*, чем масштаб)

### Мини‑пример «вручную»
Пусть  
\(u=(1,2)\), \(v=(2,1)\)

1) \(u\cdot v = 1\cdot 2 + 2\cdot 1 = 4\)  
2) \(\|u\|=\sqrt{1^2+2^2}=\sqrt{5}\), \(\|v\|=\sqrt{2^2+1^2}=\sqrt{5}\)  
3) \(\cos= 4/5 = 0.8\)

🧩 **Задачка:** посчитайте косинус для \(a=(1,0,1)\) и \(b=(0,1,1)\).  
(Потом проверим кодом.)


In [ ]:
import numpy as np

def cosine(u, v, eps=1e-12):
    u = np.asarray(u, dtype=float)
    v = np.asarray(v, dtype=float)
    return float(u @ v / (np.linalg.norm(u) * np.linalg.norm(v) + eps))

u = (1, 2)
v = (2, 1)
print("cos(u,v) =", cosine(u, v))

a = (1, 0, 1)
b = (0, 1, 1)
print("cos(a,b) =", cosine(a, b))


cos(u,v) = 0.7999999999998398
cos(a,b) = 0.49999999999974987


💡 **Что здесь считаем руками (в этом семинаре):**
1) **Косинусную близость**: скалярное произведение + нормы → \(\cos(	heta)\).  
2) (Дальше добавим) **PMI/PPMI на игрушечном примере**: из co-occurrence считаем вероятности и лог‑отношение.

Идея: вы один раз «прочувствовали» математику на маленьких числах — дальше на датасете это просто функции.

### Важная практика
- Косинусная близость ∈ [-1, 1]
- Для TF‑IDF/BoW чаще получаются значения **[0, 1]** (там нет «минусов»)
- Для эмбеддингов минусы возможны: «противоположные» направления

Дальше почти всё будет сводиться к: **собрать вектор → посчитать cosine → взять top‑k**.


## 3) Мини‑эмбеддинги «с нуля»: co-occurrence → PMI/PPMI → SVD

Это не потому что «так надо в жизни», а чтобы **руками почувствовать математику**.

### Шаг 1. Co-occurrence (счёт совместных встреч)
Берём окно ±k вокруг слова и считаем, какие слова часто «ходят рядом».

### Шаг 2. PMI / PPMI (взвешивание)
$$
PMI(w,c) = \log \frac{P(w,c)}{P(w)P(c)}
$$
PPMI — это то же самое, но отрицательные значения обнуляем:
$$
PPMI(w,c)=\max(0,PMI(w,c))
$$

### Шаг 3. SVD (сжатие)
PPMI‑матрица обычно огромная. SVD даёт нам низкоразмерные вектора.

Сделаем игрушечный корпус и пройдём весь путь.


In [ ]:
from collections import Counter, defaultdict

toy_corpus = [
    "я купил новый телефон",
    "я купил новый ноутбук",
    "новый телефон очень хороший",
    "ноутбук тоже хороший",
]

tokens = [sent.split() for sent in toy_corpus]
vocab = sorted(set(w for s in tokens for w in s))
v2i = {w:i for i,w in enumerate(vocab)}
print("Vocab:", vocab)

# co-occurrence counts with window=1
k = 1
co = np.zeros((len(vocab), len(vocab)), dtype=float)

for sent in tokens:
    for i, w in enumerate(sent):
        wi = v2i[w]
        for j in range(max(0, i-k), min(len(sent), i+k+1)):
            if j == i:
                continue
            cj = v2i[sent[j]]
            co[wi, cj] += 1

co_df = pd.DataFrame(co, index=vocab, columns=vocab)
co_df


Vocab: ['купил', 'новый', 'ноутбук', 'очень', 'телефон', 'тоже', 'хороший', 'я']


,купил,новый,ноутбук,очень,телефон,тоже,хороший,я
купил,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0
новый,2.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0
ноутбук,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
очень,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
телефон,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0
тоже,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
хороший,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
я,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### PMI/PPMI на игрушке

Тонкость: чтобы не делить на ноль, добавим маленькое ε. В реальных корпусах обычно всё плотнее, но идея та же.


In [ ]:
eps = 1e-12
total = co.sum()
Pw = co.sum(axis=1) / (total + eps)          # P(w)
Pc = co.sum(axis=0) / (total + eps)          # P(c)
Pwc = co / (total + eps)                     # P(w,c)

PMI = np.log((Pwc + eps) / (Pw[:,None]*Pc[None,:] + eps))
PPMI = np.maximum(0, PMI)

ppmi_df = pd.DataFrame(PPMI, index=vocab, columns=vocab)
ppmi_df


,купил,новый,ноутбук,очень,телефон,тоже,хороший,я
купил,0.000000,0.788457,0.000000,0.000000,0.000000,0.000000,0.000000,1.704748
новый,0.788457,0.000000,0.788457,0.000000,1.076139,0.000000,0.000000,0.000000
ноутбук,0.000000,0.788457,0.000000,0.000000,0.000000,1.704748,0.000000,0.000000
очень,0.000000,0.000000,0.000000,0.000000,1.299283,0.000000,1.704748,0.000000
телефон,0.000000,1.076139,0.000000,1.299283,0.000000,0.000000,0.000000,0.000000
тоже,0.000000,0.000000,1.704748,0.000000,0.000000,0.000000,1.704748,0.000000
хороший,0.000000,0.000000,0.000000,1.704748,0.000000,1.704748,0.000000,0.000000
я,1.704748,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#### Мини‑пример «почти вручную»: считаем PMI для одной пары

Возьмём пару слов из игрушечного корпуса (ниже в таблице co-occurrence):  
пусть \(w=\) **«купил»**, \(c=\) **«новый»**.

Напоминание:
$$
\mathrm{PMI}(w,c)=\log\frac{P(w,c)}{P(w)\,P(c)},\qquad \mathrm{PPMI}=\max(0,\mathrm{PMI})
$$

Сейчас мы **вытащим конкретные числа** \(P(w,c),P(w),P(c)\) из матрицы co-occurrence и посчитаем PMI.  
🧩 **Задачка:** после этого попробуйте сами посчитать PMI для другой пары (например, «телефон»–«новый») и сверить с кодом.


In [ ]:
# Достаём конкретные числа из нашего co-occurrence и считаем PMI/PPMI пошагово
w, c = "купил", "новый"

wi, ci = v2i[w], v2i[c]

total = co.sum()
count_wc = co[wi, ci]
Pw = co.sum(axis=1)[wi] / (total + 1e-12)
Pc = co.sum(axis=0)[ci] / (total + 1e-12)
Pwc = count_wc / (total + 1e-12)

pmi = float(np.log((Pwc + 1e-12) / (Pw * Pc + 1e-12)))
ppmi = max(0.0, pmi)

print(f"Всего совместных встреч (total) = {total:.0f}")
print(f"count({w},{c}) = {count_wc:.0f}")
print(f"P({w})  = {Pw:.4f}")
print(f"P({c})  = {Pc:.4f}")
print(f"P({w},{c}) = {Pwc:.4f}")
print(f"PMI({w},{c})  = {pmi:.4f}")
print(f"PPMI({w},{c}) = {ppmi:.4f}")

print('\nПроверка: значение из PPMI-таблицы =', float(ppmi_df.loc[w, c]))

Всего совместных встреч (total) = 22
count(купил,новый) = 2
P(купил)  = 0.1818
P(новый)  = 0.2273
P(купил,новый) = 0.0909
PMI(купил,новый)  = 0.7885
PPMI(купил,новый) = 0.7885

Проверка: значение из PPMI-таблицы = 0.7884573603511155


### SVD → вектора слов

Берём k компонент (например, 2) — получаем 2D‑вектора, чтобы можно было «потрогать».


In [ ]:
from numpy.linalg import svd

U, S, Vt = svd(PPMI, full_matrices=False)

k_dim = 2
# Простая версия: word vectors = U[:, :k] * sqrt(S[:k])
W = U[:, :k_dim] * np.sqrt(S[:k_dim])

W_df = pd.DataFrame(W, index=vocab, columns=[f"dim{i+1}" for i in range(k_dim)])
W_df


,dim1,dim2
купил,-2.065159e-01,-1.911287e-16
новый,1.075099e-16,-5.194180e-01
ноутбук,-8.622226e-01,0.000000e+00
очень,-1.015371e-16,-1.046604e+00
телефон,-6.474385e-01,0.000000e+00
тоже,-3.583663e-17,-1.258727e+00
хороший,-1.326057e+00,0.000000e+00
я,-7.167326e-17,-1.187907e-01


### Проверяем «сходство = геометрия»

Посчитаем, что ближе к слову **телефон**.


In [ ]:
def topk_sim(word, W, vocab, k=5):
    if word not in vocab:
        return None
    idx = vocab.index(word)
    u = W[idx]
    sims = []
    for j, w in enumerate(vocab):
        if j == idx:
            continue
        sims.append((w, cosine(u, W[j])))
    sims.sort(key=lambda x: x[1], reverse=True)
    return sims[:k]

topk_sim("телефон", W, vocab, k=5)


[('хороший', 0.9999999999988353),
 ('ноутбук', 0.9999999999982087),
 ('купил', 0.9999999999925209),
 ('я', 6.033575918576725e-16),
 ('очень', 9.701581865115336e-17)]

✅ Мы сделали «эмбеддинги» из счётов и линейной алгебры.  
В жизни чаще берут Word2Vec/fastText, потому что они хорошо масштабируются и дают качество на больших корпусах.


## 4) Word2Vec на нашем корпусе (gensim)

Теперь то же самое, но **prediction‑based**: модель учится предсказывать контекст и вынуждена кодировать смысл в векторах.

Две архитектуры:
- **CBOW**: контекст → центральное слово (быстрее)
- **Skip‑gram**: слово → слова контекста (лучше на редких)

Мы обучим на небольшом датасете твитов (как в семинаре 1). Да, корпус маленький, но для практики хватит.


### Загружаем датасет твитов (тот же, что в семинаре 1)

Если ссылки вдруг умерли — напишите, я дам зеркала. Но обычно Dropbox живёт.


In [ ]:
import pathlib

def download(url, out_path):
    """Пробуем скачать файл. Если интернета нет/сайт не отвечает — просто печатаем предупреждение.
    (Дальше чтение CSV всё равно упадёт в try/except и включит встроенный мини‑датасет.)
    """
    out_path = pathlib.Path(out_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    if out_path.exists() and out_path.stat().st_size > 0:
        print(f"✔ Уже скачано: {out_path}")
        return True

    # 1) пробуем wget (в Colab обычно ок)
    try:
        !wget -q -O "{out_path}" "{url}"
        if out_path.exists() and out_path.stat().st_size > 0:
            print(f"✅ Скачано wget: {out_path} ({out_path.stat().st_size/1e6:.1f} MB)")
            return True
    except Exception:
        pass

    # 2) фолбэк: requests
    try:
        import requests
        print("wget не сработал, пробуем requests...")
        r = requests.get(url, stream=True, timeout=30, allow_redirects=True)
        r.raise_for_status()
        with open(out_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=1<<20):
                if chunk:
                    f.write(chunk)
        if out_path.exists() and out_path.stat().st_size > 0:
            print(f"✅ Скачано requests: {out_path} ({out_path.stat().st_size/1e6:.1f} MB)")
            return True
    except Exception as e:
        print(f"⚠️ Не удалось скачать {out_path}: {e}")

    return False

pos_url = "https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=1"
neg_url = "https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=1"

download(pos_url, "positive.csv")
download(neg_url, "negative.csv")


✅ Скачано wget: positive.csv (26.2 MB)
✅ Скачано wget: negative.csv (24.5 MB)


True

In [ ]:
# Читаем датасет твитов. Если файлы не скачались (например, оффлайн),
# используем маленький встроенный пример, чтобы весь ноутбук можно было прогнать.

try:
    positive = pd.read_csv("positive.csv", sep=";", usecols=[3], names=["text"])
    positive["label"] = "positive"

    negative = pd.read_csv("negative.csv", sep=";", usecols=[3], names=["text"])
    negative["label"] = "negative"

    df = pd.concat([positive, negative], ignore_index=True)
    print("✅ Загружено из CSV, shape:", df.shape)

    if len(df) == 0:
        raise ValueError("CSV прочитались, но датасет пустой")

except Exception as e:
    print("⚠️ Не удалось загрузить positive.csv/negative.csv — включаю встроенный мини‑датасет.")
    print("Причина:", repr(e))

    sample_pos = [
        "Обожаю этот фильм, настроение супер!",
        "Сегодня отличный день, всё получилось 🙂",
        "Спасибо, очень помогли, класс!",
        "Кофе вкусный, сервис отличный",
        "Ура! наконец-то получилось!",
        "Как же приятно гулять вечером по городу",
        "Шикарная новость, я рад",
        "Очень круто, рекомендую всем",
        "Праздник удался, атмосфера топ",
        "Люблю такую музыку, кайф"
    ]
    sample_neg = [
        "Ужасный сервис, больше не приду",
        "Опять всё сломалось, бесит",
        "Настроение испорчено, кошмар",
        "Не работает, зря потратил время",
        "Как же плохо сделано, разочарование",
        "Доставка задержалась, очень неприятно",
        "Слишком дорого и качество так себе",
        "Совсем не понравилось, печаль",
        "Постоянные ошибки, невозможно пользоваться",
        "Никому не советую, полный провал"
    ]
    df = pd.DataFrame({
        "text": sample_pos + sample_neg,
        "label": ["positive"]*len(sample_pos) + ["negative"]*len(sample_neg)
    })
    print("✅ Встроенный мини‑датасет, shape:", df.shape)

df.sample(min(5, len(df)), random_state=42)


✅ Загружено из CSV, shape: (226834, 2)


,text,label
215081,мыс на меня обиделась:(\nя ей даже ничего не с...,negative
220754,"аааааааааааааааааааа,не хочу на работу :(",negative
60463,"У меня какой-то особенный вид ушей! :D, некото...",positive
213263,@simonovkon он неплохой человек в жизни. Я ра...,negative
113055,"RT @Darina_Lo: Домааааа\nЕхали на такси, пели ...",positive


### Мини‑чистка и токенизация (как в семинаре 1, но без фанатизма)

⚠️ Здесь мы НЕ делаем лемматизацию: для Word2Vec/fastText на малом корпусе это часто делает хуже/шумнее.  
Делаем минимально: уберём ссылки/упоминания, приведём к нижнему регистру, разобьём по словам.


In [ ]:
URL_RE = re.compile(r"http\S+|www\.\S+")
MENTION_RE = re.compile(r"@\w+")
MULTISPACE_RE = re.compile(r"\s+")

def clean_tweet(text: str) -> str:
    text = str(text)
    text = text.replace("\n", " ")
    text = URL_RE.sub(" ", text)
    text = MENTION_RE.sub(" ", text)
    text = text.lower()
    text = MULTISPACE_RE.sub(" ", text).strip()
    return text

# Простая токенизация: оставим только буквы/цифры (можно спорить, но для семинара ок)
TOKEN_RE = re.compile(r"[а-яёa-z0-9]+", re.IGNORECASE)

def tokenize_ru(text: str):
    return TOKEN_RE.findall(text.lower())

df["text_clean"] = df["text"].map(clean_tweet)
df["tokens"] = df["text_clean"].map(tokenize_ru)

df[["text_clean", "tokens", "label"]].sample(min(5, len(df)), random_state=0)


,text_clean,tokens,label
109500,rt : завтра я улетаю в москву. а послезавтра с...,"[rt, завтра, я, улетаю, в, москву, а, послезав...",positive
127808,rt : кто ненавидит пробки ретвит((((( rt,"[rt, кто, ненавидит, пробки, ретвит, rt]",negative
193322,почему? почему у меня осталось так мало времен...,"[почему, почему, у, меня, осталось, так, мало,...",negative
202883,"я уже вижу, как дети будут просматривать ваши ...","[я, уже, вижу, как, дети, будут, просматривать...",negative
172745,"сам предмет маразм( и аньч, прости, что н смог...","[сам, предмет, маразм, и, аньч, прости, что, н...",negative


### Обучаем Word2Vec (gensim)

Сделаем два варианта:
- **CBOW** (sg=0)
- **Skip‑gram** (sg=1)

Параметры под маленький корпус:
- vector_size: 100 (можно 50)
- window: 5
- min_count: 3 (слова реже — выкидываем)
- negative: 10 (негативные примеры)


In [ ]:
from gensim.models import Word2Vec

sentences = df["tokens"].tolist()

w2v_cbow = Word2Vec(
    sentences=sentences,
    vector_size=50,
    window=5,
    min_count=3,
    sg=0,          # CBOW
    negative=10,
    workers=2,
    epochs=3,
)

w2v_sg = Word2Vec(
    sentences=sentences,
    vector_size=50,
    window=5,
    min_count=3,
    sg=1,          # Skip-gram
    negative=10,
    workers=2,
    epochs=3,
)

print("Vocab size CBOW:", len(w2v_cbow.wv))
print("Vocab size SG  :", len(w2v_sg.wv))


KeyboardInterrupt: 

In [ ]:
print()

### Смотрим ближайших соседей (intrinsic‑оценка)

⚠️ На маленьком корпусе соседи будут «шумные», но идея видна.

Попробуйте подставить слова типа: `хорошо`, `плохо`, `люблю`, `ненавижу`, `класс`, `ужас`.


In [ ]:
def safe_most_similar(model, word, topn=10):
    if word not in model.wv:
        print(f"❌ '{word}' нет в словаре модели (min_count мог выкинуть).")
        return None
    return model.wv.most_similar(word, topn=topn)

safe_most_similar(w2v_sg, "хорошо", topn=10)


NameError: name 'w2v_sg' is not defined

## 5) fastText: подсловные n‑граммы и русский язык

Главная фишка: вектор слова = сумма/среднее векторов **символьных n‑грамм**.  
Поэтому:
- морфология (похожие основы) учитывается лучше
- OOV (редкие/новые слова) не «обнуляются»

Обучим fastText на том же корпусе и сравним поведение на «похожих формах».


In [ ]:
from gensim.models import FastText

ft = FastText(
    sentences=sentences,
    vector_size=100,
    window=5,
    min_count=3,
    sg=1,              # чаще берут skip-gram
    negative=10,
    workers=2,
    epochs=10,
    min_n=3, max_n=5,  # n-граммы
)

print("Vocab size fastText:", len(ft.wv))


KeyboardInterrupt: 

### Демонстрация OOV‑эффекта

Слова, которых нет в словаре Word2Vec, обычно вызывают ошибку.  
В fastText можно получить вектор даже для «почти новых» слов (если n‑граммы знакомы).

⚠️ На нашем датасете твитов примеры могут быть неидеальны, поэтому ниже есть запасной пример на «эконометрика/эконометрический/...».


In [ ]:
def try_get_vector(model, word):
    try:
        v = model.wv.get_vector(word)
        return v
    except KeyError as e:
        print(f"❌ KeyError для '{word}': {e}")
        return None

test_words = ["эконометрика", "эконометрический", "эконометрик", "эконометрики"]

print("Word2Vec (SG):")
for w in test_words:
    _ = try_get_vector(w2v_sg, w)

print("\nfastText:")
for w in test_words:
    v = ft.wv.get_vector(w)   # fastText умеет OOV
    print(w, "->", v[:5], "...")  # покажем первые 5 чисел


Если вы видите, что Word2Vec не знает этих слов — это нормально: **они не встретились** в твитах.  
fastText всё равно даст вектор, потому что соберёт его из n‑грамм.

В реальных проектах часто берут **готовые pretrained fastText** (особенно для русского).  
Но это обычно большие файлы — на семинаре тренируем маленькую модель, чтобы всё работало быстро.


## 6) Документные вектора (doc‑vector)

Мы хотим вектор не слова, а **целого текста**.

Два базовых способа:
1) **Среднее по словам**  
$$
\vec d = \frac{1}{n} \sum_{i=1}^n \vec w_i
$$
2) **TF‑IDF‑взвешенное среднее** (часто лучше)  
$$
\vec d = \frac{\sum_i \mathrm{tfidf}(w_i) \cdot \vec w_i}{\sum_i \mathrm{tfidf}(w_i)}
$$

Сделаем оба и сравним.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF на чистом тексте (строках)
tfidf_vec = TfidfVectorizer(
    tokenizer=tokenize_ru,
    preprocessor=lambda x: x,   # уже cleaned
    token_pattern=None,         # важно, когда tokenizer задан
    min_df=2
)
X_tfidf = tfidf_vec.fit_transform(df["text_clean"])

vocab_tfidf = tfidf_vec.vocabulary_
idf = tfidf_vec.idf_

# Быстрый доступ: word -> idf
idf_map = {w: idf[idx] for w, idx in vocab_tfidf.items()}
len(idf_map), list(idf_map.items())[:5]


In [ ]:
def doc_vector_mean(tokens, model):
    vecs = []
    for t in tokens:
        if t in model.wv:
            vecs.append(model.wv[t])
    if not vecs:
        return np.zeros(model.vector_size, dtype=float)
    return np.mean(vecs, axis=0)

def doc_vector_tfidf(tokens, model, idf_map):
    vecs = []
    weights = []
    for t in tokens:
        if t in model.wv:
            w = idf_map.get(t, 1.0)  # если слова нет в tfidf vocab — вес 1
            vecs.append(model.wv[t])
            weights.append(w)
    if not vecs:
        return np.zeros(model.vector_size, dtype=float)
    weights = np.asarray(weights, dtype=float)
    vecs = np.asarray(vecs, dtype=float)
    return (vecs * weights[:, None]).sum(axis=0) / (weights.sum() + 1e-12)

# Проверим на одном тексте
i = 0
tokens_i = df.loc[i, "tokens"]
d_mean = doc_vector_mean(tokens_i, w2v_sg)
d_tfidf = doc_vector_tfidf(tokens_i, w2v_sg, idf_map)
d_mean[:5], d_tfidf[:5]


## 7) Мини‑проект: семантический поиск похожих твитов

Сценарий: у нас есть один твит (запрос), хотим найти **топ‑k похожих**.

Сравним 3 подхода:
1) **TF‑IDF cosine** (из семинара 1)
2) **Word2Vec mean cosine**
3) **Word2Vec TF‑IDF‑weighted mean cosine**

⚠️ Важно: это качественная демонстрация. На маленьком корпусе результаты будут «местами странные» — это нормально.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Подготовим матрицы doc-векторов
D_mean = np.vstack([doc_vector_mean(toks, w2v_sg) for toks in df["tokens"]])
D_wt   = np.vstack([doc_vector_tfidf(toks, w2v_sg, idf_map) for toks in df["tokens"]])

# Для TF-IDF cosine будем использовать sparse-матрицу X_tfidf
print("Shapes:", X_tfidf.shape, D_mean.shape, D_wt.shape)


## 7) Визуализация: как выглядят полученные вектора (2 графика)

Ниже — два быстрых графика, чтобы **пощупать** пространство векторов:

1) **PCA проекция doc-векторов** (TF‑IDF‑взвешенное среднее Word2Vec) и раскраска по метке (positive/negative).  
2) **PCA проекция векторов слов** для небольшого списка частых токенов + подписи точек.

> Важно: PCA — это просто проекция в 2D для визуализации, а не «истинная» геометрия исходного пространства.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# --- (1) PCA для doc-векторов ---
pca = PCA(n_components=2, random_state=42)
Z = pca.fit_transform(D_wt)

plt.figure(figsize=(7, 5))
for lab in df["label"].unique():
    mask = (df["label"] == lab).values
    plt.scatter(Z[mask, 0], Z[mask, 1], s=12, alpha=0.7, label=lab)
plt.title("PCA проекция doc-векторов (TF-IDF-weighted Word2Vec)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()

# --- (2) PCA для векторов слов (небольшой список) ---
# Возьмём самые частые токены, которые точно есть в модели
from collections import Counter
cnt = Counter([t for toks in df["tokens"] for t in toks])
cand = [w for w,_ in cnt.most_common(40) if w in w2v_sg.wv]

words = cand[:25]  # чтобы график не превращался в кашу
W = np.vstack([w2v_sg.wv[w] for w in words])
Zw = PCA(n_components=2, random_state=42).fit_transform(W)

plt.figure(figsize=(7, 5))
plt.scatter(Zw[:, 0], Zw[:, 1], s=30, alpha=0.8)
for i, w in enumerate(words):
    plt.text(Zw[i, 0], Zw[i, 1], w, fontsize=9)
plt.title("PCA проекция векторов слов (Word2Vec)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()


In [ ]:
def topk_similar_texts(query_idx, k=5):
    query_text = df.loc[query_idx, "text_clean"]

    # 1) TF-IDF
    sims_tfidf = cosine_similarity(X_tfidf[query_idx], X_tfidf).ravel()
    # 2) mean docvec
    sims_mean = cosine_similarity(D_mean[query_idx].reshape(1,-1), D_mean).ravel()
    # 3) weighted docvec
    sims_wt = cosine_similarity(D_wt[query_idx].reshape(1,-1), D_wt).ravel()

    def show_top(sims, title):
        idxs = np.argsort(-sims)[:k+1]  # +1 потому что первым будет сам запрос
        print("\n" + "-"*80)
        print(title)
        print("QUERY:", query_text[:180])
        print("-"*80)
        shown = 0
        for j in idxs:
            if j == query_idx:
                continue
            print(f"[{shown+1}] sim={sims[j]:.3f} | label={df.loc[j,'label']} | {df.loc[j,'text_clean'][:180]}")
            shown += 1
            if shown >= k:
                break

    show_top(sims_tfidf, "1) TF-IDF cosine")
    show_top(sims_mean,  "2) Word2Vec mean cosine")
    show_top(sims_wt,    "3) Word2Vec TF-IDF-weighted mean cosine")

# Возьмём случайный твит (можно поменять индекс)
topk_similar_texts(query_idx=min(42, len(df)-1), k=5)


🧪 **Эксперимент:**  
- попробуйте убрать из токенов очень частые слова (типа «я», «и», «не»)  
- и посмотрите, как меняются соседи  
(спойлер: иногда качество возрастает, иногда ломается 🙂)

Мини‑подсказка: можно завести список стоп‑слов и фильтровать `df["tokens"]`.


## 8) Бонус: классификация по doc‑vector vs TF‑IDF (честное сравнение)

Сделаем так:
- одно и то же разбиение train/test
- один и тот же классификатор (LogReg)
- сравним метрики

Ожидание:
- на шумном тексте TF‑IDF часто очень силён
- doc‑vector может быть хуже на маленьком корпусе, но иногда лучше обобщает


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.preprocessing import MaxAbsScaler

X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
    df["text_clean"].values,
    df["label"].values,
    np.arange(len(df)),
    test_size=0.2,
    random_state=42,
    stratify=df["label"].values
)

print(len(X_train), len(X_test))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF baseline
vec = TfidfVectorizer(
    tokenizer=tokenize_ru,
    preprocessor=lambda x: x,
    token_pattern=None,
    ngram_range=(1,2),
    min_df=2
)

Xtr = vec.fit_transform(X_train)
Xte = vec.transform(X_test)

scaler = MaxAbsScaler()
Xtr_s = scaler.fit_transform(Xtr)
Xte_s = scaler.transform(Xte)

clf = LogisticRegression(max_iter=400, solver="saga", n_jobs=-1, random_state=42)
clf.fit(Xtr_s, y_train)

pred = clf.predict(Xte_s)
print("TF-IDF baseline")
print(classification_report(y_test, pred))


In [ ]:
# Doc-vectors: mean и weighted (на Word2Vec SG)
D_mean_train = D_mean[idx_train]
D_mean_test  = D_mean[idx_test]

D_wt_train = D_wt[idx_train]
D_wt_test  = D_wt[idx_test]

def train_logreg_dense(Xtr, Xte, ytr, yte, title):
    clf = LogisticRegression(max_iter=400, solver="lbfgs", n_jobs=-1, random_state=42)
    clf.fit(Xtr, ytr)
    pred = clf.predict(Xte)
    print(title)
    print(classification_report(yte, pred))

train_logreg_dense(D_mean_train, D_mean_test, y_train, y_test, "DocVec mean (Word2Vec)")
train_logreg_dense(D_wt_train, D_wt_test, y_train, y_test, "DocVec TF-IDF-weighted (Word2Vec)")


## Итоги и мини‑домашка

**Что сделали:**
- руками посчитали **cosine similarity**
- собрали «эмбеддинги из линейной алгебры»: co-occurrence → PPMI → SVD
- обучили Word2Vec и fastText на своём корпусе
- собрали **doc‑vector** (mean и TF‑IDF‑weighted)
- сделали мини‑семантический поиск и сравнили с TF‑IDF
- (бонус) сравнили классификацию TF‑IDF vs doc‑vector

### Мини‑домашка (на 20–40 минут)
1) Возьмите 10 разных твитов и для каждого найдите топ‑5 соседей:
   - TF‑IDF
   - Word2Vec mean
   - Word2Vec weighted
   Сравните глазами: где «смысл» лучше?
2) Попробуйте обучить Word2Vec на **Skip‑gram** с `min_count=2` и `vector_size=50` — что меняется?
3) (чуть сложнее) Добавьте стоп‑слова и проверьте, что происходит с соседями.

Если будет желание — на следующей паре обсудим, почему контекстные эмбеддинги (BERT) делают это всё ещё лучше 🙂
